# Importation des modules

In [1]:
#Pour le web scrapping
import requests
from bs4 import BeautifulSoup

# Pour paralléliser le web scrapping
from loky import ProcessPoolExecutor
from itertools import chain

# Pour les commandes basiques
import pandas as pd
import numpy as np
from collections import Counter

# Sur le site `pukanina.com`

In [2]:
response = requests.get("https://www.pukanina.com/faq-foire-aux-questions-infos-pratiques/")
QandA = BeautifulSoup(response.text).select(".et_pb_toggle")

questions = [elem.h5.text for elem in QandA]
answers = [elem.div.text[1:] for elem in QandA]

# Sur le site `perou.org`

**Définition de fonctions**

In [3]:
# Cette fonction prend en argument un des urls FAQ du site perou.org et renvoie une liste de tuples
# dont le premier élément est une question et le second est la réponse associé à cette question.
def webscraper_decouvrir_le_perou(url):
    response = requests.get(url)
    BS1 = (BeautifulSoup(response.text)
          .select("body > h3"))
    BS2 = (BeautifulSoup(response.text)
          .select("body > blockquote"))
    n = len(BS1)
    
    return [(BS1[i].text,BS2[i].text) for i in range(n)]

# Cette fonction permet de paralléliser la fonction précendente afin de réduire le temps de calcul.
def all_informations(key):
    with ProcessPoolExecutor() as epool:
        mapped_values = epool.map(webscraper_decouvrir_le_perou, key)
    return list(chain(*mapped_values))

In [4]:
%%time
liste_urls = ["https://perou.org/faq/trip.php",
             "https://perou.org/faq/health.php",
             "https://perou.org/faq/transports.php",
             "https://perou.org/faq/security.php",
             "https://perou.org/faq/live_peru.php"]

QandA = all_informations(liste_urls)

Wall time: 10.1 s


In [5]:
for q,a in QandA:
    questions.append(q.replace(" \r\n ","").strip())
    answers.append(a.replace('(adsbygoogle = window.adsbygoogle || []).push({});',"").replace("\r\n","").strip())

# Sur le site `perou-sur-mesure.com`

**Définition de fonction**

In [6]:
# Cette fonction prend en entrée une liste de mots (ex : ['h3','p','p','p','h3','p']) et renvoie une liste
# qui indique le nombre de 'p' entre le début de la liste et le premier 'h3' (ce sera toujours 0), entre chaque 'h3'
# et entre le dernier 'h3' et la fin de la liste (ex : [0,3,1]).
# Cette fonction nous aide à associer chaque réponse à sa question car une réponse peut être divisée en plusieurs
# paragraphes 'p'.
def count_p_between_h3(liste):
    count = 0
    new_liste = []
    for elem in liste:
        if elem=="h3":
            new_liste.append(count)
            count = 0
        else:
            count+=1
    new_liste.append(count)
    return np.cumsum(new_liste)

In [7]:
response = requests.get("https://www.perou-sur-mesure.com/infos-pratiques#/")
QandA = BeautifulSoup(response.text).select(".Faq-questions__content > div > div")

for elem in QandA:
    nb_of_p = count_p_between_h3([i.name for i in elem if i.name is not None][1:])
    questions += [i.text for i in elem.select("h3")]
    p = [i.text for i in elem.select("p")]
    answers += [" ".join(p[nb_of_p[i]:nb_of_p[i+1]]) for i in range(len(nb_of_p)-1)]

# Ajout des thèmes

**Première idée : 11 thèmes**
```python
dico_themes = {1:"Voyage",
          2:"Administratif",
          3:"Argent",
          4:"Santé",
          5:"Communication",
          6:"Bagages",
          7:"Hébergement",
          8:"Transport",
          9:"Trek/Visite/Guide",
          10:"Vivre au Pérou/Culture",
          11:"Sécurité"}
liste_themes = [1,1,2,1,1,9,3,9,9,1,3,2,2,11,4,4,4,4,4,4,4,4,5,5,5,5,6,6,6,6,6,3,3,3,3,3,7,7,7,8,8,8,8,8,9,9,9,9,
                1,9,7,1,1,11,6,11,2,3,9,1,9,4,4,8,8,8,8,8,8,11,11,11,11,10,10,10,10,10,
                1,1,2,2,2,2,2,2,2,3,7,7,7,8,8,8,8,5,6,4,4,11,11,3,10,10,10,10]

themes = [dico_themes[elem] for elem in liste_themes]
```

**Deuxième idée : 8 thèmes**

In [8]:
dico_themes = {1:"Voyage",
          2:"Administratif/Argent",
          3:"Administratif/Argent",
          4:"Santé/Sécurité",
          5:"Communication",
          6:"Bagages",
          7:"Hébergement",
          8:"Transport",
          9:"Trek/Visite/Guide",
          10:"Vivre au Pérou/Culture",
          11:"Santé/Sécurité"}

liste_themes = [1,1,2,1,1,9,3,9,9,1,3,2,2,11,4,4,4,4,4,4,4,4,5,5,5,5,6,6,6,6,6,3,3,3,3,3,7,7,7,8,8,8,8,8,9,9,9,9,
                1,9,7,1,1,11,6,11,2,3,9,1,9,4,4,8,8,8,8,8,8,11,11,11,11,10,10,10,10,10,
                1,1,2,2,2,2,2,2,2,3,7,7,7,8,8,8,8,5,6,4,4,11,11,3,10,10,10,10]

themes = [dico_themes[elem] for elem in liste_themes]
Counter(themes)

Counter({'Voyage': 11,
         'Administratif/Argent': 21,
         'Trek/Visite/Guide': 10,
         'Santé/Sécurité': 21,
         'Communication': 5,
         'Bagages': 7,
         'Hébergement': 7,
         'Transport': 15,
         'Vivre au Pérou/Culture': 9})

# Tableau de données

In [9]:
df = pd.DataFrame(data = {'Themes':themes,'Questions': questions, 'Answers': answers})

# Nous décidons d'enlever les questions et réponses portant sur le thème "Communication"
# car celui-ci ne comporte que 5 questions.
df = df[df.Themes != 'Communication']

In [10]:
df.head()

,Themes,Questions,Answers
0,Voyage,Pour et contre d'un voyage groupé en tout peti...,Nous programmons des voyages groupés à dates f...
1,Voyage,Pour et contre d'un voyage sur mesure/individu...,Avantages:\n\nun circuit individuel est modifi...
2,Administratif/Argent,Pourquoi vous n'incluez pas les commissions ba...,Nous n’incluons pas les commissions bancaires ...
3,Voyage,Pourquoi vous limitez les voyages groupés à 6 ...,Par souci de maintenir la qualité d’un voyage ...
4,Voyage,Quelle est la meilleure période pour voyager ?,Nous avons dans nos latitudes en gros 2 saison...


In [11]:
df.tail()

,Themes,Questions,Answers
101,Administratif/Argent,Y a t-il des réductions pour les enfants ?,Souvent les hébergements proposent des réducti...
102,Vivre au Pérou/Culture,Quels types de souvenirs rapporter d’un voyage...,Pour partager avec vos proches toutes les merv...
103,Vivre au Pérou/Culture,Que nous réserve la cuisine péruvienne ?,La cuisine péruvienne est aussi variée que la ...
104,Vivre au Pérou/Culture,Lexique Espagnol,"Les basiques : Oui : Sí\nNon : No\nBonjour, sa..."
105,Vivre au Pérou/Culture,Lexique Quechua :,Bonjour (matin) : Allin p’unchay\nOui : Arí\nD...


# Nettoyage des données

In [12]:
# On remarque dans certaines réponses, il y a des mots qui sont séparés par plusieurs espaces.
# La fonction suite permet de résoudre ce problème. Elle prend en entrée une phrase et renvoie cette même phrase
# mais nettoyée.
def cleaning(text):
    return text.replace("     ",' ').replace('   ',' ')

df.Answers = df.Answers.apply(cleaning)

# Exportation

In [13]:
df.to_csv('../Data/Q_A.csv', sep = ';', index = False, encoding = 'utf-8', na_rep = 'NA')